# Least Recently Used (LRU) Cache

The usual implementation of an `LRUCache` uses a dictionary for storing nodes and retrieving them with O(1) access, and a doubly linked list for easily moving nodes around as they are accessed and pruning nodes when capacity has been exceeded.

In [43]:
from __future__ import annotations
from dataclasses import dataclass
from typing import Any


@dataclass
class Node:
    key: Any = None
    value: Any = None
    prev: Node = None
    next: Node = None


class LRUCache:
    def __init__(self, capacity):
        self.capacity = capacity
        self.nodes = {}
        self.root = self.tail = None
        
    def put(self, key, value):
        if key in self.nodes:
            self.nodes[key].value = value
            self._move_to_end(key)
            return
        
        if self.tail is None:
            node = self.tail = self.root = Node(key, value)
        else:
            node = self.tail.next = self.tail = Node(key, value, self.tail)
            
        self.nodes[key] = node

        if len(self.nodes) > self.capacity:
            del self.nodes[self.root.key]

            self.root.next.prev = None
            self.root = self.root.next

    def get(self, key):
        if key not in self.nodes:
            raise KeyError(f'{key} not found')

        self._move_to_end(key)
        return self.nodes[key].value

    def _move_to_end(self, key):
        node = self.nodes[key]
        
        if node == self.tail:
            return

        if node == self.root:
            self.root = node.next
        else:
            node.prev.next = node.next
        
        node.prev = self.tail
        node.next = None
        self.tail.next = node
        self.tail = node


A simplier implementation can use `OrderedDict` which acts like both a dictionary and a linked list.

In [44]:
from collections import OrderedDict
 
    
class LRUCacheOrderedDict:
    def __init__(self, capacity):
        self.cache = OrderedDict()
        self.capacity = capacity
        
    def put(self, key, value):
        self.cache[key] = value
        self.cache.move_to_end(key)
        
        if len(self.cache) > self.capacity:
            self.cache.popitem(last=False)

    def get(self, key) -> int:
        if key not in self.cache:
            raise KeyError(f'{key} not found')

        self.cache.move_to_end(key)
        return self.cache[key]

In [46]:
import unittest


class Tests:        
    def test_put_and_get(self):
        self.cache.put(1, 'one')
        self.assertEqual(self.cache.get(1), 'one')

    def test_moving_keys(self):
        self.cache.put(1, 'one')
        
        for _ in range(3):
            self.assertEqual(self.cache.get(1), 'one')
        
    def test_missing_key(self):
        # Keys that don't exist will raise a KeyError
        with self.assertRaises(KeyError):
            self.cache.get('pretend')

    def test_eviction(self):
        # Add 900 items
        for i in range(900):
            self.cache.put(i, i)

        # The first 895 get evicted.
        for i in range(895):
            with self.assertRaises(KeyError):
                self.cache.get(i)

        # The last five remain.
        for i in range(895, 900):
            self.assertEqual(self.cache.get(i), i)
                
    def test_eviction_with_moving_keys(self):
        # Add five items
        self.cache.put(1, 1)
        self.cache.put(2, 2)
        self.cache.put(3, 3)
        self.cache.put(4, 4)
        self.cache.put(5, 5)
        
        # Add 1 again to move it to make it most recently used
        self.cache.put(1, 1)
        
        # Add additional item, one more than the capacity.
        self.cache.put(6, 6)

        # That evicted 2.
        self.assertEqual(self.cache.get(1), 1)
        with self.assertRaises(KeyError):
            self.cache.get(2)
            
    
class TestLRUCache(unittest.TestCase, Tests):
    def setUp(self):
        self.cache = LRUCache(5)
        

class TestLRUCacheOrderedDict(unittest.TestCase, Tests):
    def setUp(self):
        self.cache = LRUCacheOrderedDict(5)


unittest.main(argv=[''], exit=False)

..........
----------------------------------------------------------------------
Ran 10 tests in 0.014s

OK
